In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import traceback
import os
import pandas as pd

# === CONFIGURACIÓN INICIAL ===
chrome_driver_path = "C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\chromedriver.exe"  # Asegúrate que el archivo exista aquí
download_dir = "C:\\Users\\yonat\\OneDrive\\Documentos\\programas"  # Cambia si quieres otro destino
usuario_okta = "yonatan.osorio@openenglish.com"
password_okta = "Matias2023#"
okta_url = "https://openeducation.okta.com"

# Configurar Chrome con ruta de descarga
options = webdriver.ChromeOptions()
options.add_argument("--start-minizied")
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

# === BLOQUE TRY PARA EL LOGIN AUTOMÁTICO ===
try:
    print("🚀 Iniciando navegador...")
    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)

    print("🔐 Navegando a Okta...")
    driver.get(okta_url)

    # Esperar que aparezca el campo de usuario y completar credenciales
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "input28"))).send_keys(usuario_okta)
    driver.find_element(By.ID, "input36").send_keys(password_okta)
    driver.find_element(By.XPATH, '//input[@value="Iniciar sesión"]').click()
    print("✅ Login exitoso en Okta. Esperando acceso a la app de Salesforce...")

    # Espera a que aparezca el ícono de Salesforce y haz clic automáticamente
    salesforce_app_xpath = '//span[contains(text(), "Salesforce")]/ancestor::a'
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, salesforce_app_xpath))).click()
    print("🔗 Ingreso exitoso a salesforce")

    try:
    # Aquí pegas el link directo del reporte exportable (CSV)
        report_url = "https://openeducation.my.salesforce.com/00OKV000008PVfQ?export=1&enc=UTF-8&xf=xlsx"

        print("📄 Navegando al reporte de Salesforce...")
        driver.get(report_url)

        # Esperar que comience la descarga
        time.sleep(5)  # Dale tiempo a que se descargue el archivo

        print("📥 Esperando que se descargue el archivo...")

        # Verifica si el archivo ya está en la carpeta
        archivo_descargado = None
        timeout = 30
        tiempo_espera = 0
        while tiempo_espera < timeout:
            for archivo in os.listdir(download_dir):
                if archivo.endswith(".csv") and not archivo.endswith(".crdownload"):
                    archivo_descargado = os.path.join(download_dir, archivo)
                    break
            if archivo_descargado:
                break
            time.sleep(1)
            tiempo_espera += 1

        if archivo_descargado:
            print(f"✅ Reporte descargado correctamente: {archivo_descargado}")
        else:
            print("⚠️ No se detectó ningún archivo descargado.")
            driver.quit()

        # === CONVERTIR CSV A EXCEL ===
        excel_path = archivo_descargado.replace(".csv", ".xlsx")
        df = pd.read_csv(archivo_descargado, sep=",", encoding="utf-8")
        df.to_excel(excel_path, index=False)

        print(f"✅ CSV convertido a Excel: {excel_path}")

        # Aquí puedes continuar con la parte 4: actualizar otro Excel

    except Exception as e:
        print(f"⚠️ Error durante la ejecución: {e}")
    finally:
        driver.quit()



except Exception as e:
    print("❌ Ocurrió un error durante el login en Okta:")
    traceback.print_exc()
    driver.quit()


🚀 Iniciando navegador...
🔐 Navegando a Okta...
✅ Login exitoso en Okta. Esperando acceso a la app de Salesforce...
🔗 Ingreso exitoso a salesforce
📄 Navegando al reporte de Salesforce...
📥 Esperando que se descargue el archivo...
✅ Reporte descargado correctamente: C:\Users\yonat\OneDrive\Documentos\programas\report1744415399353.csv
⚠️ Error durante la ejecución: No columns to parse from file


In [ ]:


# === PARTE 2: Acceder al reporte y descargarlo ===



📄 Navegando al reporte de Salesforce...
⚠️ Error durante la ejecución: HTTPConnectionPool(host='localhost', port=57028): Max retries exceeded with url: /session/507441f524f0a0e2dba3c0499260996f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000011483B0F350>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))
